In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.metrics import classification_report, confusion_matrix
from IPython.display import Image, display
from sklearn.metrics import plot_confusion_matrix
from sklearn import tree
from sklearn import utils
from sklearn.metrics import mean_squared_error
from sklearn import tree



import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import graphviz
import matplotlib.pyplot as plt
import sys
import string
import csv
import random
import copy
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
def randomForest(clfs,estimators,featuresFracs,crit):
    for criter in crit:
        for estimator in estimators:
            for featuresFrac in featuresFracs:   
                  clfs.append(RandomForestClassifier(
                      n_estimators = estimator,
                      #integer, optional (default=100)
                      criterion= criter,
                      #string, optional (default=”gini”)
                      max_depth= None,
                      #integer or None, optional (default=None)
                      min_samples_split=2,
                      #int, float, optional (default=2)
                      min_samples_leaf=1,
                      #int, float, optional (default=1)
                      min_weight_fraction_leaf=0.0,
                      #float, optional (default=0.)
                      max_features= featuresFrac,
                      #int, float, string or None, optional (default=”auto”)
                      max_leaf_nodes=None,
                      #int or None, optional (default=None)
                      min_impurity_decrease=0.01,
                      #float, optional (default=0.)
                      bootstrap=True,
                      #boolean, optional (default=True)
                      oob_score=False,
                      #bool (default=False)
                      n_jobs=None,
                      #int or None, optional (default=None)
                      random_state=None,
                      #int, RandomState instance or None, optional (default=None)
                      verbose=0,
                      #int, optional (default=0)
                      warm_start=False,
                      #bool, optional (default=False)
                      ccp_alpha=0.0,
                      #non-negative float, optional (default=0.0)
                      max_samples=None
                      #int or float, default=None
                      ))

In [ ]:
def decisionTree(clfs, crits, impDecreases):    
    for crit in crits:
        for imp in impDecreases:
            clf = tree.DecisionTreeClassifier(
                  criterion= "entropy",
                  splitter= crit,
                  max_depth=None,
                  min_samples_split= 2,
                  min_samples_leaf=5,
                  min_weight_fraction_leaf=0.0,
                  max_features=None,
                  random_state=None,
                  max_leaf_nodes=None,
                  min_impurity_decrease=imp,
                  class_weight=None,
                  ccp_alpha = 0.0)
            clfs.append(clf)

In [ ]:
def moreImportant(importance):
  arrSorted = copy.deepcopy(importance)
  arrSorted.sort()
  arrSorted = arrSorted[::-1]
  indxs = []
  for i in range(0,5):
    temp = np.where(arrSorted[i] == importance)
    temp = int(temp[0])
    indxs.append(temp)
  
  return indxs

In [ ]:
def removeAditiveName(names):
#     names = {x.replace('Miss', ' ').replace('Mr', ' ').replace('Mrs', ' ').replace('Dr', ' ').replace('Capt', ' ').replace('Master', ' ').replace('Mme', ' ').replace('Mlle', ' ').replace('Sir', ' ').replace('Col', ' ').replace('Rev', ' ') for x in names}
#     names = {x.replace('  ', ' ') for x in names}
    newNames = []
    for name in names:
        name = removePunctuation(name)
        newNames.append(name)
    return newNames


In [ ]:
def removePunctuation(name):
    newName = ''
    for ch in name:
        if ch not in string.punctuation:
            newName = newName + ch 
    
    return newName

In [ ]:
def removeNoNumber(name):
    newName = ''
    digits = '0123456789'
    for ch in name:
        if ch in digits:
            newName = newName + ch
    if(newName == ''):
        return 0
    return float(newName)

In [ ]:
def printAll(value):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(value)

In [ ]:
def plotImportance(forest, X, y):
    forest.fit(X, y)
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X.shape[1]):
        print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

    # Plot the feature importances of the forest
    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(X.shape[1]), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.xticks(range(X.shape[1]), indices)
    plt.xlim([-1, X.shape[1]])
    plt.show()

In [ ]:
#Read the csv and extract the data in labels,X and y
X = pd.read_csv('/kaggle/input/titanic/train.csv')
y = X['Survived']
X = X.drop(columns="Survived")

#Preprocessing to convert string data in numerical data

#Select all no numerical Data
procX = X.select_dtypes(include=[object])

#Fill NaN for numerical data with 0
for column in X.columns:
    columnName = str(column)
    X[columnName].fillna(0, inplace = True)
#Fill NaN for numerical data with 0, in this case only Cabin and Embarked has NaN     
procX['Cabin'].fillna('0', inplace = True)
procX['Embarked'].fillna('0', inplace = True)

#Create the preprocessing Label encoder, apply it to the no numerical data and replace the result of the last step in ...
# the original data frame (X)
le = preprocessing.LabelEncoder()
procX = procX.apply(le.fit_transform)
X['Name'] = procX['Name']
X['Sex'] = procX['Sex']
X['Ticket'] = procX['Ticket']
X['Cabin'] = procX['Cabin']
X['Embarked'] = procX['Embarked']

#Train a decision tree to 'estimate' the age of passengers without this feature based on the other ones

#Extracting the data for train and test the decision tree 
y_age = X['Age'][X['Age'] != 0]
y_age = y_age.tolist()      
X_age = X[X['Age'] != 0]
X_age = X_age.drop(columns = 'Age')
X_age.insert(1, 'Survived', y[X['Age'] != 0])

#Split train and test data
X_train, X_test, y_train, y_test = train_test_split(X_age, y_age, test_size=0.30, random_state=None)

#Set the test values for variables in tree (criterio and minImpurity Decrease)
clfs = []
crits = ['random']
impDecreases = [0.1]
decisionTree(clfs, crits, impDecreases)
best = []

#Split train and test data
X_train, X_test, y_train, y_test = train_test_split(X_age, y_age, test_size=0.30, random_state=None)

#Change all set's features to multiclass instead of continuos
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)
a,b =  X_train.shape
for row in range(0, a):
    X_train.iloc[row] = le.fit_transform(X_train.iloc[row])
a,b =  X_test.shape    
for row in range(0, a):
    X_test.iloc[row] = le.fit_transform(X_test.iloc[row])

#Train
testTrees = []
for clf in clfs:
    tempClf = clf.fit(X_train, y_train)
    testTrees.append(tempClf)
    y_pred = tempClf.predict(X_test)

#Fill the rows with NaN in age column 
emptyAge =  X[X['Age'] == 0]
emptyAge = emptyAge.drop(columns = 'Age')
emptyAge.insert(1, 'Survived', y[X['Age'] == 0])
predAge = clf.predict(emptyAge)
X['Age'][X['Age'] == 0] = predAge

#Split train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=None)

#Train and test the tree classifier 
scores = []
forests = []
estimators = [1000]
crit = ['gini']
featuresFracs = [0.99]
randomForest(forests,estimators,featuresFracs,crit)
importance = []
print(len(trees))
for clf in trees:
    clf.fit(X_train, y_train)
    scores.append(clf.score( X_test, y_test))
    importance.append(clf.feature_importances_)
print(scores)
